In [19]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime
import json
import warnings
from unidecode import unidecode

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'
    # path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [20]:
years =  list(range(2014, 2022 + 1))

In [21]:
races = {
        'season': [],
        'round': [],
        'circuit_id': [],
        'country': [],
        'lat': [],
        'long': [],
        'url': []
        }

for year in years:
    url = 'https://ergast.com/api/f1/{}.json'
    r = requests.get(url.format(year))
    json = r.json()

    for item in json['MRData']['RaceTable']['Races']:
        races['season'].append(int(item['season']))
        races['round'].append(int(item['round']))
        races['circuit_id'].append(item['Circuit']['circuitId'])
        races['country'].append(item['Circuit']['Location']['country'])    
        races['lat'].append(float(item['Circuit']['Location']['lat']))
        races['long'].append(float(item['Circuit']['Location']['long']))
        races['url'].append(item['url'])

races = pd.DataFrame(races)
races.head()

,season,round,circuit_id,country,lat,long,url
0,2014,1,albert_park,Australia,-37.84970,144.96800,http://en.wikipedia.org/wiki/2014_Australian_G...
1,2014,2,sepang,Malaysia,2.76083,101.73800,http://en.wikipedia.org/wiki/2014_Malaysian_Gr...
2,2014,3,bahrain,Bahrain,26.03250,50.51060,http://en.wikipedia.org/wiki/2014_Bahrain_Gran...
3,2014,4,shanghai,China,31.33890,121.22000,http://en.wikipedia.org/wiki/2014_Chinese_Gran...
4,2014,5,catalunya,Spain,41.57000,2.26111,http://en.wikipedia.org/wiki/2014_Spanish_Gran...


In [22]:
### Query API

schedules = {
        'season': [],
        'round': [],
        'circuit_id': [],
        'date': []
        }

for year in years:

    url = 'http://ergast.com/api/f1/{}.json'
    r = requests.get(url.format(year))
    json = r.json()

    for item in json['MRData']['RaceTable']['Races']:
        schedules['season'].append(int(item['season']))
        schedules['round'].append(int(item['round']))
        schedules['circuit_id'].append(item['Circuit']['circuitId'])
        schedules['date'].append(datetime.strptime(item['date']+ 'T' +item['time'], '%Y-%m-%dT%H:%M:%SZ'))

schedules = pd.DataFrame(schedules)
schedules.head()

,season,round,circuit_id,date
0,2014,1,albert_park,2014-03-16 06:00:00
1,2014,2,sepang,2014-03-30 08:00:00
2,2014,3,bahrain,2014-04-06 15:00:00
3,2014,4,shanghai,2014-04-20 07:00:00
4,2014,5,catalunya,2014-05-11 12:00:00


In [23]:
race_schedules = pd.merge(races, schedules, how='inner', on=['season', 'round', 'circuit_id'])

race_schedules.to_csv(path+'data/races.csv', index=False)

In [24]:
races = pd.read_csv(path+'data/races.csv')

rounds = []
for year in np.array(races.season.unique()):
    rounds.append([year, list(races[races.season == year]['round'])])

In [25]:
# query API
    
results = {
    'season': [],
    'round':[],
    'circuit_id':[],
    'driver': [],
    'date_of_birth': [],
    'nationality': [],
    'constructor': [],
    'grid': [],
    'time': [],
    'status': [],
    'points': [],
    'podium': [],
    'fastest_lap': []
    }

for n in list(range(len(rounds))):
    for i in rounds[n][1]:
    
        url = 'http://ergast.com/api/f1/{}/{}/results.json'
        r = requests.get(url.format(rounds[n][0], i))
        json = r.json()

        for item in json['MRData']['RaceTable']['Races'][0]['Results']:

            results['season'].append(int(json['MRData']['RaceTable']['Races'][0]['season']))
            results['round'].append(int(json['MRData']['RaceTable']['Races'][0]['round']))
            results['circuit_id'].append(json['MRData']['RaceTable']['Races'][0]['Circuit']['circuitId'])
            results['driver'].append(
                unidecode(item['Driver']['givenName'].lower()) +'_'+ unidecode(item['Driver']['familyName'].lower())
                )
            results['date_of_birth'].append(item['Driver']['dateOfBirth'])
            results['nationality'].append(item['Driver']['nationality'])
            results['constructor'].append(item['Constructor']['constructorId'])
            results['grid'].append(int(item['grid']))

            try:
                results['time'].append(int(item['Time']['millis']))
            except:
                results['time'].append(None)

            results['status'].append(item['status'])
            results['points'].append(float(item['points']))
            results['podium'].append(int(item['position']))

            try:
                results['fastest_lap'].append(item['FastestLap']['Time']['time'])
            except:
                results['fastest_lap'].append(None)


results = pd.DataFrame(results)
results.head()

,season,round,circuit_id,driver,date_of_birth,nationality,constructor,grid,time,status,points,podium,fastest_lap
0,2014,1,albert_park,nico_rosberg,1985-06-27,German,mercedes,3,5578710.0,Finished,25.0,1,1:32.478
1,2014,1,albert_park,kevin_magnussen,1992-10-05,Danish,mclaren,4,5605487.0,Finished,18.0,2,1:33.066
2,2014,1,albert_park,jenson_button,1980-01-19,British,mclaren,10,5608737.0,Finished,15.0,3,1:32.917
3,2014,1,albert_park,fernando_alonso,1981-07-29,Spanish,ferrari,5,5613994.0,Finished,12.0,4,1:33.186
4,2014,1,albert_park,valtteri_bottas,1989-08-28,Finnish,williams,15,5626349.0,Finished,10.0,5,1:32.616


In [26]:
results.to_csv(path+'data/results.csv', index=False)

In [28]:
driver_standings = {
    'season': [],
    'round':[],
    'driver': [],
    'driver_points': [],
    'driver_wins': [],
    'driver_standings_pos': []}

for n in list(range(len(rounds))):
    for i in rounds[n][1]:
    
        url = 'https://ergast.com/api/f1/{}/{}/driverStandings.json'
        r = requests.get(url.format(rounds[n][0], i))
        json = r.json()

        for item in json['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings']:
            driver_standings['season'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['season']))
            driver_standings['round'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['round']))    
            driver_standings['driver'].append(
                unidecode(item['Driver']['givenName'].lower()) +'_'+ unidecode(item['Driver']['familyName'].lower())
                )
            driver_standings['driver_points'].append(float(item['points']))
            driver_standings['driver_wins'].append(int(item['wins']))
            driver_standings['driver_standings_pos'].append(int(item['position']))

driver_standings = pd.DataFrame(driver_standings)
driver_standings.tail(21)

,season,round,driver,driver_points,driver_wins,driver_standings_pos
3868,2022,22,charles_leclerc,308.0,3,2
3869,2022,22,sergio_perez,305.0,2,3
3870,2022,22,george_russell,275.0,1,4
3871,2022,22,carlos_sainz,246.0,1,5
3872,2022,22,lewis_hamilton,240.0,0,6
3873,2022,22,lando_norris,122.0,0,7
3874,2022,22,esteban_ocon,92.0,0,8
3875,2022,22,fernando_alonso,81.0,0,9
3876,2022,22,valtteri_bottas,49.0,0,10
3877,2022,22,daniel_ricciardo,37.0,0,11


In [29]:
driver_standings.to_csv(path+'data/driver_standings.csv', index=False)

In [30]:
constructor_rounds = rounds

constructor_standings = {'season': [],
                    'round':[],
                    'constructor': [],
                    'constructor_points': [],
                    'constructor_wins': [],
                   'constructor_standings_pos': []
                   }

for n in list(range(len(constructor_rounds))):
    for i in constructor_rounds[n][1]:
    
        url = 'https://ergast.com/api/f1/{}/{}/constructorStandings.json'
        r = requests.get(url.format(constructor_rounds[n][0], i))
        json = r.json()

        for item in json['MRData']['StandingsTable']['StandingsLists'][0]['ConstructorStandings']:
            constructor_standings['season'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['season']))
            constructor_standings['round'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['round']))
            constructor_standings['constructor'].append(item['Constructor']['constructorId'])
            constructor_standings['constructor_points'].append(float(item['points']))
            constructor_standings['constructor_wins'].append(int(item['wins']))
            constructor_standings['constructor_standings_pos'].append(int(item['position']))
            
constructor_standings = pd.DataFrame(constructor_standings)
constructor_standings.tail(10)

,season,round,constructor,constructor_points,constructor_wins,constructor_standings_pos
1849,2022,22,red_bull,759.0,17,1
1850,2022,22,ferrari,554.0,4,2
1851,2022,22,mercedes,515.0,1,3
1852,2022,22,alpine,173.0,0,4
1853,2022,22,mclaren,159.0,0,5
1854,2022,22,alfa,55.0,0,6
1855,2022,22,aston_martin,55.0,0,7
1856,2022,22,haas,37.0,0,8
1857,2022,22,alphatauri,35.0,0,9
1858,2022,22,williams,8.0,0,10


In [31]:
constructor_standings.to_csv(path+'data/constructor_standings.csv', index=False)